In [ ]:
# Detect and identify charging events of ride-hailing EV, with dividing waiting and charging time. This code is based on RAPIDS (cuDF & cuPy)

In [1]:
import cudf
import cupy as cp
from sqlalchemy import create_engine, text
import sqlalchemy
import pandas as pd
import os
from urllib.parse import quote_plus
import subprocess
import psycopg2
import numpy as np
import pyarrow as pa
from scipy.spatial import cKDTree
import folium
from folium.plugins import MarkerCluster
import pyarrow.parquet as pq
from heapq import heappush, heappop
import dask_cudf
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter
import logging, shutil
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Parmaeters

In [2]:
# Thresholds for stay events detection
V_THR = 5   # (km/h) Max speed to be considered
T_GAP_THR =600      # (s) Max time gap between consecutive points in the same stay
D_GAP_THR = 100  # (m) Max distance between points to break a stay segment
STAY_THR = 600       # (s) Minimum duration for a sequence of points to be considered a valid stay
D_THR_M = 200  #(m) Threshold to consider a stay as a charging event

In [3]:
# Database connection
username = 'xuhang.liu'
password = 'xuhangLIU@HOMES'
hostname = 'homes-database.epfl.ch'
port = '30767'
dbname = 'Shenzhen_Taxi'
password = quote_plus(password)
DB_URL = f"postgresql://{username}:{password}@{hostname}:{port}/{dbname}"

# Pre-defined fucntions

In [4]:
# Haversine Distance Function
def haversine_distance_gpu(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in meters between two points
    on the earth using cupy for GPU acceleration.
    """
    # cupy operations require float64 for precision in trig functions
    lon1_rad = cp.radians(lon1.astype(cp.float64))
    lat1_rad = cp.radians(lat1.astype(cp.float64))
    lon2_rad = cp.radians(lon2.astype(cp.float64))
    lat2_rad = cp.radians(lat2.astype(cp.float64))

    # Haversine formula
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = cp.sin(dlat / 2)**2 + cp.cos(lat1_rad) * cp.cos(lat2_rad) * cp.sin(dlon / 2)**2
    c = 2 * cp.arcsin(cp.sqrt(a))
    # Radius of earth in meters.
    R = 6371000
    return c * R

# Data loading

In [ ]:
# Save data locally
table_name = "taxi_data2020_01_01"
save_path = os.path.join(os.getcwd(), 'gps_data_01_01_v5.csv')
if  not os.path.exists(save_path):
    query = f"""
    SELECT
        gid, taxiid,
        to_char(time, 'YYYY-MM-DD HH24:MI:SS.US') as time,
        lon, lat, velocity, angle, passenger, zoneid, validity,
        hvalue, "OSM_edgeid", "OSM_distance"
    FROM {table_name}
    WHERE  passenger = 0 AND velocity<={V_THR}  AND time >= '2020-01-01'
    ORDER BY taxiid, time
    """

    conn = psycopg2.connect(DB_URL)
    cur = conn.cursor()
    print('data loading...')
    with open(save_path, 'w', encoding='utf-8') as f:
        cur.copy_expert(f"COPY ({query}) TO STDOUT WITH CSV HEADER", f)

    cur.close()
    conn.close()
    print('data loaded.')
else:
    print('data have been saved locally')

In [ ]:
table_name = "taxi_data2020_01_01"
save_path2 = os.path.join(os.getcwd(), 'gps_data_01_01_all.csv')
if  not os.path.exists(save_path2):
    query = f"""
    SELECT
        gid, taxiid,
        to_char(time, 'YYYY-MM-DD HH24:MI:SS.US') as time,
        lon, lat, velocity, angle, passenger, zoneid, "OSM_distance"
    FROM {table_name}
    WHERE  time >= '2020-01-01'
    ORDER BY taxiid, time
    """

    conn = psycopg2.connect(DB_URL)
    cur = conn.cursor()
    print('data loading...')
    with open(save_path2, 'w', encoding='utf-8') as f:
        cur.copy_expert(f"COPY ({query}) TO STDOUT WITH CSV HEADER", f)

    cur.close()
    conn.close()
    print('data loaded.')
else:
    print('data have been saved locally')

data loading...


In [ ]:
# Read data as df

par_path = 'gps_data_01_01_v5.parquet'
if os.path.exists(par_path):
    print("load parquet file")
    gdf = cudf.read_parquet(par_path)
else:
    cols_to_keep = ['taxiid', 'time', 'lon', 'lat','velocity', 'zoneid','angle','OSM_edgeid', 'OSM_distance']
    data_path = 'gps_data_01_01_v5.csv'
    CHUNK_SIZE = 10_000_000  # data size of each reading
    list_of_chunks = [] # save data
    offset = 0
    header_read = False
    column_names = None

    # Main Reading Loop
    while True:
        try:
            if not header_read:
                header_df = cudf.read_csv(data_path, nrows=0)
                column_names = header_df.columns.to_list()
                header_read = True
                offset = 1
            # read data
            chunk = cudf.read_csv(
                data_path,
                nrows=CHUNK_SIZE,
                skiprows=offset,
                header=None,
                names=column_names,
                dtype={
                    'taxiid': 'str',
                    'lon': np.float32,
                    'lat': np.float32,
                    'angle': np.int8,
                    'zoneid': np.int8,
                    'validity': np.int8
                },
                parse_dates=['time']
            )

            if len(chunk) == 0:
                print("have read all data, concatenating:")
                break

            list_of_chunks.append(chunk)
            offset += len(chunk)
            print(f" have read {offset-1} data")
        except Exception as e:
            print(f"error in reading: {e}")
            break

    # Concatenate
    if list_of_chunks:
        gdf = cudf.concat(list_of_chunks, ignore_index=True)
        print(f"get dataframe with columns of {len(gdf)}")
        del list_of_chunks
    else:
        print("no data")
        gdf = cudf.DataFrame()
    # remove NaN value
    gdf = gdf.dropna()

In [ ]:
!nvidia-smi

In [ ]:
# convert taxiid to category to save resource
gdf['taxiid'] = gdf['taxiid'].astype('category')

In [ ]:
# save dataframe
par_path = 'gps_data_01_01_v5.parquet'
gdf.to_parquet(par_path,engine='pyarrow')

# Identify stay events

In [ ]:
# Calculate gaps within each taxi's trajectory
if not gdf.empty:
    # Get previous point's time and coordinates
    gdf['prev_time'] = gdf.groupby('taxiid')['time'].shift()
    gdf['prev_lon'] = gdf.groupby('taxiid')['lon'].shift().fillna(0)
    gdf['prev_lat'] = gdf.groupby('taxiid')['lat'].shift().fillna(0)

    # Calculate time gap between two consecutive points(unit:second)
    gdf['dt'] = (gdf['time'] - gdf['prev_time']).dt.seconds.fillna(99999)

    # Calculate distance between two consecutive points(unit:second)
    gdf['dd'] = haversine_distance_gpu(gdf['lon'], gdf['lat'], gdf['prev_lon'], gdf['prev_lat'])


In [ ]:
if not gdf.empty:
    # Flag the start of a stay, the first point always correspond a stay segment
    stay_flags = (gdf['dt'] > T_GAP_THR) | (gdf['dd'] > D_GAP_THR) | (gdf['prev_time'].isnull())

    # Assign a unique ID to each stay and convert the boolean series to a cupy array for cumsum
    gdf['stay_tag'] = cp.asarray(stay_flags).astype(cp.int32).cumsum()

In [ ]:
!nvidia-smi

In [ ]:
gdf = gdf[['taxiid', 'stay_tag', 'time', 'lon', 'lat']]

In [ ]:
# Aggregate by stay tags to find stays
if not gdf.empty:
    #Group by tag and taxi
    agg_stays = gdf.groupby(['taxiid', 'stay_tag']).agg(
        start_time=('time', 'min'),
        end_time=('time', 'max'),
        stay_lon=('lon', 'mean'),
        stay_lat=('lat', 'mean'),
        points=('taxiid', 'count')
    ).reset_index()

    #  Calculate duration and filter for valid stays
    agg_stays['duration_s'] = (agg_stays['end_time'] - agg_stays['start_time']).dt.seconds
    final_stays = agg_stays[agg_stays['duration_s'] >= STAY_THR].copy()
    final_stays = final_stays.sort_values(['taxiid', 'start_time'])
    print(f"Found {len(final_stays)} valid stay events.")


# Matching with stations


In [ ]:
# station and piles data
STA_CSV_PATH = 'station_information.csv'
stations_df = pd.read_csv(STA_CSV_PATH,dtype={'station_id': np.int32})
PILE_CSV_PATH = 'pile_rated_power.csv'
piles_df = pd.read_csv(PILE_CSV_PATH,dtype={'station_id': np.int32,'pile_id': np.int32})
# filter piles (0 < power <= max_power) ---
max_power =60
valid_piles_df = piles_df[(piles_df['power'] > 0) & (piles_df['power'] <= max_power)].copy()
valid_piles_df['low_power_piles'] = (valid_piles_df['power'] < 10).astype(int)
valid_piles_df['high_power_piles'] = (valid_piles_df['power'] >= 10).astype(int)

In [ ]:
powers = valid_piles_df['power'].astype('float32')

fig, ax = plt.subplots(figsize=(8, 5))

bins = np.arange(0, powers.max() + 5, 10)
sns.histplot(
    powers,
    bins=bins,
    color='#377eb8',
    edgecolor='black',
    alpha=0.8,
    ax=ax,
    stat='percent'
)
ax.yaxis.set_major_formatter(FormatStrFormatter('%.0f%%'))
ax.set_yticks(np.arange(0, ax.get_ylim()[1]+1, 10))
ax.set_xlabel('Charger Power (kW)')
ax.set_ylabel('percent')
ax.set_title('Distribution of Charger Rated Power')

plt.tight_layout()
plt.show()

In [ ]:
valid_piles_df.head()

In [ ]:
stat_pile_df = valid_piles_df.groupby('station_id').agg(
        num_LP_piles=('low_power_piles', 'sum'),
        num_HP_piles=('high_power_piles', 'sum'),
        num_dist_powers=('power', 'nunique'),
        num_piles =('power','count'),
        max_power =('power','max'),
        min_power =('power','min'),
        avg_power =('power','mean')
    ).reset_index()
stat_pile_df.head()

In [ ]:
test = stat_pile_df['num_piles'].isna().any()
test

In [ ]:
# stations and piles info vis
sns.set_theme(style="whitegrid")
stat_pile_df['pile_conf'] = np.select(
    condlist=[
        (stat_pile_df['num_LP_piles'] > 0) & (stat_pile_df['num_HP_piles'] == 0),
        (stat_pile_df['num_HP_piles'] > 0) & (stat_pile_df['num_LP_piles'] == 0),
        (stat_pile_df['num_LP_piles'] > 0) & (stat_pile_df['num_HP_piles'] > 0)
    ],
    choicelist=['Slow charging', 'Fast charging', 'Hybrid'],
    default='Other'
)

type_counts = stat_pile_df['pile_conf'].value_counts().reindex(
    ['Slow charging', 'Fast charging', 'Hybrid'])

In [ ]:
# pile configuration vis
colors = ['#4daf4a', '#377eb8', '#ff7f00']   # Slow, Fast, Mixed
fig, ax = plt.subplots(figsize=(4,4))
wedges, texts, autotexts = ax.pie(
    type_counts.values,
    labels=type_counts.index,
    autopct='%1.1f%%',
    startangle=10,
    counterclock=False,
    colors=colors,
    pctdistance=0.7,
    textprops={'fontsize': 10}
)
centre_circle = plt.Circle((0,0), 0.45, fc='white')
for text in texts:
    text.set_color('black')
for autotext in autotexts:
    autotext.set_color('black')

slow_charging_label_index = 0
if 'Slow charging' in type_counts.index:
    slow_charging_label_index = list(type_counts.index).index('Slow charging')

slow_label = texts[slow_charging_label_index]
slow_autopct = autotexts[slow_charging_label_index]
slow_label.set_position((-0.3, -1.05))
slow_label.set_ha('center')

fig.gca().add_artist(centre_circle)
#ax.set_title('Station piles configuration', fontsize=14)
fig.patch.set_alpha(1)
plt.tight_layout()
plt.show()

In [ ]:
# slow charging proportion in hybrid stations
mixed_df = stat_pile_df[stat_pile_df['pile_conf'] == 'Hybrid'].copy()
mixed_df['share_LP'] = (
    mixed_df['num_LP_piles'] / (mixed_df['num_LP_piles'] + mixed_df['num_HP_piles'])
)

fig, axes = plt.subplots(figsize=(6,4))
# KDE
sns.kdeplot(
    data=mixed_df,
    x='share_LP',
    fill=True,
    color='#ff7f00',
    ax=axes
)
axes.set_xlabel('Proportion of low-power piles')
axes.set_xlim(0,1)
axes.grid(False)
axes.xaxis.set_major_formatter(FormatStrFormatter('%g'))
#axes.set_title('KDE of Slow-Charger Share')
#sns.despine(offset=10, trim=True)
plt.tight_layout()
plt.show()

In [ ]:

import matplotlib.ticker as mtick

mixed_df = stat_pile_df[stat_pile_df['pile_conf'] == 'Hybrid'].copy()
mixed_df['share_LP'] = (
    mixed_df['num_LP_piles'] /
    (mixed_df['num_LP_piles'] + mixed_df['num_HP_piles'])
)
total_mixed = len(mixed_df)

bins = np.linspace(0,1, 11)         # 0–1 分成 10 段
fig, ax = plt.subplots(figsize=(6, 5))

counts, bin_edges, patches = ax.hist(
    mixed_df['share_LP'],
    bins=bins,
    weights=np.ones(total_mixed) / total_mixed,
    color='pink',
    edgecolor='black',
    alpha=0.9
)

ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.set_xlabel('Proportion of low-power piles')
ax.set_ylabel('Percentage in hybrid stations')
#ax.set_title('Slow-Charger Share among Mixed Stations')
ax.grid(False)
ax.set_xlim(0, 1)
ax.set_xticks(np.arange(0, 1.1, 0.1))

for frac, left, right in zip(counts, bin_edges[:-1], bin_edges[1:]):
    if frac == 0:
        continue
    x_pos = (left + right) / 2
    pct_text = f'{frac*100:.0f}%'
    ax.text(
        x_pos, frac + 0.003,
        pct_text,
        ha='center', va='bottom',
        fontsize=10,
    )
plt.tight_layout()
plt.show()

In [ ]:
# number pile power types
power_diversity_counts = (
    stat_pile_df['num_dist_powers']
      .value_counts()
      .sort_index()
)

plt.figure(figsize=(4,4))
labels = [f'{k} types' for k in power_diversity_counts.index]
plt.pie(
    power_diversity_counts.values,
    labels=labels,
    autopct='%1.1f%%',
    startangle=90,
    radius=1.1,
    counterclock=False,
    textprops={'fontsize': 10}
)
#plt.title('Distribution of piles power across stations',fontsize=11)
plt.tight_layout()
plt.show()

# most of charging piles about 95% are slow-charging(<10KW), and most stations only have slow-charing piles, evne the hybrid stations, most(>85%) of them have slow-charging piles accounting for more than 50%.  The conclusion is slow-charging is dominant

In [ ]:
#  Refine Station data
try:
    original_station_id_dtype = stations_df['station_id'].dtype
    core_station_columns = ['station_id', 'longitude', 'latitude', 'TAZID']
    core_station_columns = [col for col in core_station_columns if col in stations_df.columns]

    # stat_pile_df = stat_pile_stats.groupby('station_id').agg(
    #     low_power_piles=('low_power_piles', 'sum'),
    #     high_power_piles=('high_power_piles', 'sum')
    # ).reset_index()
    temp_stations_df = stations_df.copy()
    temp_counts_df = stat_pile_df.copy()

    if temp_stations_df['station_id'].dtype != temp_counts_df['station_id'].dtype:
        temp_stations_df['station_id'] = temp_stations_df['station_id'].astype(int)
        temp_counts_df['station_id'] = temp_counts_df['station_id'].astype(int)

    stations_df_cleaned = temp_stations_df[core_station_columns]

    # match station id
    stations_df_updated = pd.merge(
        stations_df_cleaned,
        temp_counts_df,
        on='station_id',
        how='right'
    )

    # # update piles number
    # stations_df_updated['charge_count'] = stations_df_updated['low_power_piles'] + stations_df_updated['high_power_piles']
    # # only keep stations with at least 1 valid pile
    # stations_df_updated = stations_df_updated[ stations_df_updated['charge_count']>0]

    stations_df = stations_df_updated

except FileNotFoundError:
    print(f"no {PILE_CSV_PATH}")
except Exception as e:
    print(f"error: {e}")


In [ ]:

# --- 2. Build cKDTree from Station Coordinates (on CPU) ---
# This assumes both GPS and station data use the WGS-84 coordinate system.
station_coords = stations_df[['longitude', 'latitude']].to_numpy()
station_tree = cKDTree(station_coords)

In [ ]:
#  match stay events with stations
if not final_stays.empty:

    # Transfer stay-point coordinates from GPU to CPU
    stay_coords = final_stays[['stay_lon', 'stay_lat']].to_numpy()
    #Perform the nearest neighbor query using the cKDTree
    distances_deg, indices = station_tree.query(stay_coords, k=1)

    #  Convert distance from degrees to approximate meters.
    distances_m = distances_deg * 111320

    # Transfer the numpy arrays from CPU back to GPU
    final_stays['distance_to_station'] = distances_m

    # Map the indices from the query result back to the original station_id
    station_id_lookup = stations_df['station_id'].to_numpy()
    nearest_station_ids = station_id_lookup[indices]
    final_stays['nearest_station_id'] = cudf.Series(nearest_station_ids, index=final_stays.index)

    # Filter for charging events within distance threshold ---
    charging_events = final_stays[final_stays['distance_to_station'] <= D_THR_M].copy()
    charging_events = charging_events.sort_values(['taxiid', 'start_time'])


In [4]:
os.getcwd()

In [15]:
mypath="/home/lxhep/etaxi/data/taxi_trajectories.parquet/taxiid=UUUB0C0M7"

In [16]:
mydata = pd.read_parquet(mypath)

In [17]:
mydata.head(10)

,time,lon,lat,velocity,passenger
0,2020-01-01 00:00:43,114.098412,22.543447,0,1
1,2020-01-01 00:01:28,114.098412,22.543447,0,1
2,2020-01-01 00:01:58,114.098618,22.543428,0,1
3,2020-01-01 00:02:13,114.098618,22.543428,0,1
4,2020-01-01 00:03:13,114.098618,22.543428,13,1
5,2020-01-01 00:04:13,114.098930,22.543468,0,1
6,2020-01-01 00:04:58,114.098930,22.543468,0,1
7,2020-01-01 00:05:15,114.099663,22.543573,17,1
8,2020-01-01 00:06:00,114.099579,22.546421,23,1
9,2020-01-01 00:06:43,114.099556,22.548313,42,1


In [4]:
stay_par_path = 'stay_01_01_v5.parquet'
char_par_path = 'char_01_01_v5.parquet'


In [3]:
final_stays.to_parquet(stay_par_path,engine='pyarrow')
charging_events.to_parquet(char_par_path,engine='pyarrow')

NameError: name 'final_stays' is not defined

In [5]:
if os.path.exists(stay_par_path):
    final_stays = pd.read_parquet(stay_par_path)
    print(f'stay parquet loaded')
if os.path.exists(char_par_path):
    charging_events = pd.read_parquet(char_par_path)
    print(f'char parquet loaded')

stay parquet loaded
char parquet loaded


In [10]:
unique_ids= charging_events['taxiid'].unique().tolist()

In [12]:
len(unique_ids)

19496

In [13]:
np.save('unique_ids.npy', np.array(unique_ids))

# Queue simulation and waiting time calculation

In [ ]:
def que_station(df):
    """
    Simulates the queue for a single station. The input 'df' is for one station and sorted by 'start_time'.
    """
    if df.empty:
        print('input df is empty')
        return None

    k = int(df['num_piles'].iloc[0])
    busy_chargers_heap = []  # A min-heap storing the end_time of cars currently charging
    results = []

    for row in df.itertuples():
        arrival_time = row.start_time
        departure_time = row.end_time

        # Remove chargers that have become free before the current car arrives
        while busy_chargers_heap and busy_chargers_heap[0] <= arrival_time:
            heappop(busy_chargers_heap)

        # Check for available chargers
        if len(busy_chargers_heap) < k:
            # There is a free charger, no waiting time
            wait_seconds = 0
            charge_start_time = arrival_time
        else:
            # All chargers are busy, find the earliest time a charger becomes free.
            earliest_free_time = heappop(busy_chargers_heap)
            wait_seconds = (earliest_free_time - arrival_time).total_seconds()
            charge_start_time = earliest_free_time

        # Determine if the driver gave up
        stay_duration = (departure_time - arrival_time).total_seconds()
        gave_up = (wait_seconds >= stay_duration)

        # Calculate actual charge duration and update charger status
        if gave_up:
            actual_charge_seconds = 0
            # If the car gave up, the charger it was waiting for is free again at its scheduled time
            if 'earliest_free_time' in locals():
                 heappush(busy_chargers_heap, earliest_free_time)
        else:
            actual_charge_seconds = (departure_time - charge_start_time).total_seconds()
            # This car occupies a charger until its departure time
            heappush(busy_chargers_heap, departure_time)

        results.append({
            'station_id': row.nearest_station_id,
            'taxiid': row.taxiid,
            'arrive_time': arrival_time,
            'leave_time': departure_time,
            'wait_dur': wait_seconds,
            'giveup': gave_up,
            'charge_dur': actual_charge_seconds
        })

    return pd.DataFrame(results)

In [ ]:
# Queue simulation
char_que_df = pd.DataFrame()
if 'charging_events' in locals() and not charging_events.empty:

    # Select necessary columns and convert to pandas
    queue_input_df = charging_events[[
        'nearest_station_id', 'taxiid', 'start_time', 'end_time']].to_pandas()

    # Merge with station info
    queue_input_df = queue_input_df.merge(
        stations_df[['station_id', 'num_piles']],
        left_on='nearest_station_id',
        right_on='station_id',
        how='left'
    )

    # # --- 2. Save to Parquet for efficient batch processing ---
    # QUEUE_INPUT_PARQUET = 'charging_events_for_queue_sim.parquet'
    # queue_input_df.to_parquet(QUEUE_INPUT_PARQUET)
    DST_PARQUET = 'char_que_analysis.parquet'
    # --- Main Batch Processing Loop ---
    writer = None

    queue_input_df = queue_input_df.sort_values(['nearest_station_id', 'start_time'], kind='mergesort')
    all_station_results = [
            que_station(group)
            for _, group in queue_input_df.groupby('nearest_station_id', sort=False)]
    # Concatenate results and write to the output Parquet file
    if all_station_results:
        char_que_df = pd.concat(all_station_results, ignore_index=True)
        table = pa.Table.from_pandas(char_que_df)
        if writer is None:
            writer = pq.ParquetWriter(DST_PARQUET, table.schema)
        writer.write_table(table)
    if writer:
        writer.close()
    print(f" Queue simulation finished. Results saved to '{DST_PARQUET}'.")
else:
    print("'charging_events' DataFrame not found or is empty. Cannot proceed with queue simulation.")


In [ ]:
# basic data property
avg_wait_time = char_que_df['wait_dur'].mean()
giveup_rate = char_que_df['giveup'].mean()
num_veh = char_que_df['taxiid'].nunique()
print(f"\nAverage waiting time: {avg_wait_time:.2f} seconds")
print(f"Give-up rate: {giveup_rate:.2%}")
print(f'{len(charging_events)} charging events from {num_veh} taxis')

# Taxi trajectory data

In [79]:
TMP_DIR   = 'tmp_tracks'
DS_ENERGY = 'energy_gap.parquet'
DS_PICK   = 'post_charge_pickup.parquet'
os.makedirs(TMP_DIR, exist_ok=True)


username = 'xuhang.liu'
password = 'xuhangLIU@HOMES'
hostname = 'homes-database.epfl.ch'
port = '30767'
dbname = 'Shenzhen_Taxi'
password = quote_plus(password)
DB_URL = f"postgresql://{username}:{password}@{hostname}:{port}/{dbname}"
table_name = "taxi_data2020_01_01"

BATCH_DL_THREADS = 8
V_THR = 5
CAP_KWH = 57
AVG_SPEED_KMH, CONS_KWH_PER_KM = 25, 0.18
IDLE_KW = 0
MAX_PICK_MIN = 30

# connection to sql
def sql_engine(uri=DB_URL):
    return create_engine(uri, pool_pre_ping=True, pool_size=BATCH_DL_THREADS)


# download taxi trajectory data
def download_one(tid: str, dst: str, engine, tab_name: str):
    if os.path.exists(dst):
        return dst
    sql_copy = f"""
        COPY (
          SELECT taxiid,time,lon,lat,velocity,passenger
            FROM {tab_name}
           WHERE taxiid = '{tid}'
             AND time >= '2020-01-01'
           ORDER BY time
        ) TO STDOUT WITH CSV
    """
    tmp_csv = dst + '.csv'
    with engine.begin() as sa_conn:
        pg_conn = sa_conn.connection
        with open(tmp_csv, 'w', encoding='utf-8') as f, pg_conn.cursor() as cur:
            cur.copy_expert(sql_copy, f)

    cudf.read_csv(
        tmp_csv,
        names=['taxiid','time','lon','lat','velocity','passenger'],
        dtype={'taxiid':'str','lon':'f4','lat':'f4','velocity':'f4','passenger':'i1'},
        parse_dates=['time']
    ).to_parquet(dst)
    os.remove(tmp_csv)
    return dst

# Energy consumption

In [165]:
def energy_between_charges(track: cudf.DataFrame, ce: cudf.DataFrame) -> cudf.DataFrame:
    track = track.sort_values('time')

    # ① 连续差分并填 NA，确保无掩码
    track['lon_prev'] = track['lon'].shift().fillna(track['lon'])
    track['lat_prev'] = track['lat'].shift().fillna(track['lat'])

    track['dt'] = (track['time'] - track['time'].shift()).dt.seconds
    track['dt'] = track['dt'].fillna(0).astype('int64')

    track['dist'] = haversine_distance_gpu(track['lon'], track['lat'],
                                           track['lon_prev'], track['lat_prev'])

    # filter junmp
    mask_jump = (track['dt'] <= 60) & (track['dist'] > 200)
    track = track[~mask_jump]

    track['idle'] = (track['velocity'] <= V_THR).astype('i1')

    ce = ce.sort_values('start_time')
    ce['prev_end'] = ce['end_time'].shift()

    out = []
    for r in ce.to_pandas().itertuples(index=False):
        if pd.isna(r.prev_end):           # 第一条充电无前区间
            continue
        seg = track[(track['time'] > r.prev_end) & (track['time'] <= r.start_time)]

        drive_s = int(seg.loc[seg.idle == 0, 'dt'].sum())
        idle_s  = int(seg.loc[seg.idle == 1, 'dt'].sum())
        dist_km = seg['dist'].sum() / 1000
        energy  = dist_km * CONS_KWH_PER_KM + idle_s * IDLE_KW

        out.append({
            'taxiid'         : r.taxiid,
            'charge_start'   : r.start_time,
            'prev_end'       : r.prev_end,
            'gap_s'          : drive_s + idle_s,
            'drive_s'        : drive_s,
            'idle_s'         : idle_s,
            'distance_km'    : dist_km,
            'energy_used_kWh': energy
        })
    return cudf.DataFrame(out)


In [166]:
def post_charge_pickup(track: cudf.DataFrame,
                       ce: cudf.DataFrame,
                       max_pick_min: int = 10) -> cudf.DataFrame:
    """
    计算每次充电结束后 max_pick_min 分钟内是否接客，
    返回 wait_min 与直线距离 dist_m。
    """
    # ---------- 1. 预处理轨迹 ----------
    track = track.sort_values('time')
    hired = track[track.passenger > 0]         # 载客点
    if hired.empty or ce.empty:
        return cudf.DataFrame()                # 无数据则直接返回空表

    # ---------- 2. 转 pandas（充电事件通常几十条，CPU 迭代即可） ----------
    hired_pdf = hired.to_pandas()              # 小数据，CPU 处理更灵活
    ce_pdf    = ce.to_pandas()

    res = []
    for c in ce_pdf.itertuples(index=False):
        win_end = c.end_time + pd.Timedelta(minutes=max_pick_min)
        tmp = hired_pdf[(hired_pdf.time > c.end_time) & (hired_pdf.time <= win_end)]
        if tmp.empty:
            res.append({'taxiid': c.taxiid,
                        'charge_end': c.end_time,
                        'found': False})
            continue

        pick = tmp.iloc[0]                     # 第一条载客
        wait_min = (pick.time - c.end_time).total_seconds() / 60
        dist_m   = float(
            haversine_distance_gpu(
                cp.array([c.stay_lon]), cp.array([c.stay_lat]),
                cp.array([pick.lon]),   cp.array([pick.lat])
            )[0]
        )

        res.append({'taxiid'     : c.taxiid,
                    'charge_end' : c.end_time,
                    'pickup_time': pick.time,
                    'wait_min'   : wait_min,
                    'dist_m'     : dist_m,
                    'found'      : True})

    return cudf.DataFrame(res)


In [172]:
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s | %(levelname)s | %(message)s")
engine = sql_engine()

ce_all = cudf.read_parquet(char_par_path)
taxis_all  = ce_all.taxiid.unique().to_arrow().to_pylist()
logging.info(f'Total vehicles: {len(taxis_all)}')


2025-11-10 19:13:56,213 | INFO | Total vehicles: 19496


In [173]:
taxis=taxis_all[3:4]
taxis

['UUUB0C0Q5']

In [174]:
os.makedirs(TMP_DIR, exist_ok=True)
os.makedirs(DS_ENERGY, exist_ok=True)
os.makedirs(DS_PICK,   exist_ok=True)

In [175]:
total  = len(taxis)
bar = tqdm(total=total, desc='Processing', unit='car')
with ThreadPoolExecutor(max_workers=BATCH_DL_THREADS) as pool:
    fut_map = {pool.submit(download_one, tid,
                           f'{TMP_DIR}/{tid}.parquet', engine, table_name): tid
               for tid in taxis}

    for fut in as_completed(fut_map):
        tid = fut_map[fut]
        try:
            path = fut.result()                       # 下载 + 转 Parquet
            track = cudf.read_parquet(path)
            ce_t  = ce_all[ce_all.taxiid == tid]

            energy_df = energy_between_charges(track, ce_t)
            pickup_df = post_charge_pickup(track, ce_t)

            if not energy_df.empty:
                pq.write_to_dataset(energy_df.to_arrow(), DS_ENERGY,
                                    partition_cols=['taxiid'])
            if not pickup_df.empty:
                pq.write_to_dataset(pickup_df.to_arrow(), DS_PICK,
                                    partition_cols=['taxiid'])

            logging.info(f'{tid} ✓ done')
        except Exception as e:
            logging.error(f'{tid} failed: {e}')
        finally:
            bar.update(1)
            cp._default_memory_pool.free_all_blocks()
            # try:
            #     os.remove(path)
            # except Exception:
            #     pass

bar.close()
logging.info('All done.')

Processing:   0%|          | 1/19496 [01:33<508:02:56, 93.82s/car]
2025-11-10 19:15:40,517 | INFO | All done.


In [176]:
TID   = 'UUUB0C0Q5'
DIR_E = f'energy_gap.parquet/taxiid={TID}'
DIR_P = f'post_charge_pickup.parquet/taxiid={TID}'


energy_df = pq.ParquetDataset(DIR_E).read_pandas().to_pandas()
print('consumption:',energy_df)


pickup_df = pq.ParquetDataset(DIR_P).read_pandas().to_pandas()
print('post:',pickup_df)

consumption:           charge_start            prev_end   gap_s  drive_s  idle_s  \
0  2020-01-01 07:01:26 2020-01-01 04:23:40    8619     1183    7436   
1  2020-01-01 09:47:26 2020-01-01 07:20:53    7322     6769     553   
2  2020-01-03 00:25:35 2020-01-01 10:26:56  121570    38725   82845   
3  2020-01-03 05:39:57 2020-01-03 00:40:48   16691     3445   13246   
4  2020-01-03 12:15:27 2020-01-03 06:29:27   16544     9411    7133   
5  2020-01-04 06:29:24 2020-01-03 12:33:04   57331    21677   35654   
6  2020-01-05 06:36:46 2020-01-04 06:41:38   71741    31589   40152   
7  2020-01-06 07:02:52 2020-01-05 06:48:34   77436    25738   51698   
8  2020-01-06 22:55:18 2020-01-06 07:17:24   46836    27147   19689   
9  2020-01-07 04:56:28 2020-01-06 23:16:18   18510     4666   13844   
10 2020-01-07 06:44:58 2020-01-07 05:18:43    4273     1538    2735   

    distance_km  energy_used_kWh  
0     10.307915         1.855425  
1     24.018485         4.323327  
2    295.093654        53.116

In [177]:

# ⚙️ 想查看的 taxiid


# 1️⃣ 过滤并按时间排序（GPU cuDF）
ce_tid = charging_events[charging_events.taxiid == TID] \
           .sort_values('start_time')

print(f'Found {len(ce_tid)} charging events for {TID}')

ce_tid

Found 12 charging events for UUUB0C0Q5


,taxiid,stay_tag,start_time,end_time,stay_lon,stay_lat,points,duration_s,distance_to_station,nearest_station_id
4032375,UUUB0C0Q5,2731,2020-01-01 04:11:55,2020-01-01 04:23:40,114.123706,22.567409,15,705,60.081880,2208
10108106,UUUB0C0Q5,2737,2020-01-01 07:01:26,2020-01-01 07:20:53,114.107730,22.569469,38,1167,120.196760,2101
7442462,UUUB0C0Q5,2743,2020-01-01 09:47:26,2020-01-01 10:26:56,114.061849,22.518453,72,2370,97.994113,1923
14544185,UUUB0C0Q5,2910,2020-01-03 00:25:35,2020-01-03 00:40:48,114.123779,22.567329,25,913,72.014360,2208
14347474,UUUB0C0Q5,2918,2020-01-03 05:39:57,2020-01-03 06:29:27,114.113932,22.534414,129,2970,138.370543,2143
10949258,UUUB0C0Q5,2951,2020-01-03 12:15:27,2020-01-03 12:33:04,114.020027,22.532431,31,1057,183.203650,1643
9044234,UUUB0C0Q5,3021,2020-01-04 06:29:24,2020-01-04 06:41:38,114.013931,22.545993,33,734,113.087207,1616
6361262,UUUB0C0Q5,3152,2020-01-05 06:36:46,2020-01-05 06:48:34,114.104235,22.552713,38,708,145.967162,2094
3904164,UUUB0C0Q5,3262,2020-01-06 07:02:52,2020-01-06 07:17:24,114.069562,22.526544,27,872,192.937863,2003
9424026,UUUB0C0Q5,3342,2020-01-06 22:55:18,2020-01-06 23:16:18,114.115179,22.539300,22,1260,184.194632,2145


In [178]:
st_tid = final_stays[final_stays.taxiid == TID] \
           .sort_values('start_time')

print(f'Found {len(st_tid)} stay events for {TID}')

st_tid

Found 30 stay events for UUUB0C0Q5


,taxiid,stay_tag,start_time,end_time,stay_lon,stay_lat,points,duration_s,distance_to_station,nearest_station_id
4032375,UUUB0C0Q5,2731,2020-01-01 04:11:55,2020-01-01 04:23:40,114.123706,22.567409,15,705,60.081880,2208
8730334,UUUB0C0Q5,2733,2020-01-01 04:56:43,2020-01-01 05:20:58,114.123061,22.611212,67,1455,205.938181,2225
10108106,UUUB0C0Q5,2737,2020-01-01 07:01:26,2020-01-01 07:20:53,114.107730,22.569469,38,1167,120.196760,2101
7442462,UUUB0C0Q5,2743,2020-01-01 09:47:26,2020-01-01 10:26:56,114.061849,22.518453,72,2370,97.994113,1923
3806538,UUUB0C0Q5,2792,2020-01-02 06:20:19,2020-01-02 06:41:04,114.111741,22.597628,52,1245,265.598726,2142
1471595,UUUB0C0Q5,2814,2020-01-02 10:57:21,2020-01-02 11:10:04,113.989063,22.527947,25,763,821.825154,1560
5494153,UUUB0C0Q5,2841,2020-01-02 14:02:06,2020-01-02 14:17:51,113.989038,22.527635,20,945,855.331730,1560
16769201,UUUB0C0Q5,2854,2020-01-02 16:31:51,2020-01-02 16:49:36,114.122949,22.611162,25,1065,216.518544,2190
14544185,UUUB0C0Q5,2910,2020-01-03 00:25:35,2020-01-03 00:40:48,114.123779,22.567329,25,913,72.014360,2208
14347474,UUUB0C0Q5,2918,2020-01-03 05:39:57,2020-01-03 06:29:27,114.113932,22.534414,129,2970,138.370543,2143


In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(energy_df['distance_km'], bins=30, kde=True)
plt.xlabel('Distance between charges (km)')
plt.title(f'{TID}: Distance distribution')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.scatterplot(data=pickup_df[pickup_df['found']],
                x='charge_end', y='wait_min')
plt.ylabel('Wait minutes after charging')
plt.xticks(rotation=45)
plt.title(f'{TID}: Post-charge pickup waiting time')
plt.tight_layout(); plt.show()

In [ ]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=[energy_df['stay_lat'].mean(),
                         energy_df['stay_lon'].mean()],
               zoom_start=12)
cluster = MarkerCluster().add_to(m)
for _, r in pickup_df[pickup_df['found']].iterrows():
    folium.CircleMarker(
        [r['stay_lat'], r['stay_lon']],
        radius=5,color='red',fill=True,fill_opacity=0.6,
        popup=f"Wait {r['wait_min']:.1f} min, {r['dist_m']:.0f} m"
    ).add_to(cluster)
m

In [133]:
# ⚙️ 参数：想看的 taxiid
TID = 'UUUB0C0M7'              # 改成任意车辆编号
PARQUET_PATH = f'{TMP_DIR}/{TID}.parquet'   # 若已删除，请重新 download_one 后再运行

# 1️⃣ 读取整车轨迹
if not os.path.exists(PARQUET_PATH):
    raise FileNotFoundError("先调用 download_one 或确保 Parquet 文件存在")

track = cudf.read_parquet(PARQUET_PATH) \
            .sort_values('time') \
            .to_pandas()                 # 转 pandas 便于 folium

print(f'{len(track):,} points loaded for {TID}')

# 2️⃣ 取中心点
center_lat = track['lat'].mean()
center_lon = track['lon'].mean()

# 3️⃣ Folium 地图
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=[center_lat, center_lon],
               zoom_start=11,
               tiles='OpenStreetMap')

# 3-a 轨迹折线（可选：分颜色段）
coords = track[['lat','lon']].values.tolist()
folium.PolyLine(coords, color='blue', weight=2, opacity=0.6,
                tooltip=f'{TID} trajectory').add_to(m)

# 3-b 标记起点终点
folium.Marker(coords[0], icon=folium.Icon(color='green'),
              tooltip='Start').add_to(m)
folium.Marker(coords[-1], icon=folium.Icon(color='red'),
              tooltip='End').add_to(m)

m

KeyError: 'dist'

In [ ]:
CAP_KWH          = 57          # battery capacity (kWh)
AVG_SPEED_KMH    = 25          # avg speed (km/h)
CONS_KWH_PER_KM  = 0.18        # consumption rate (kWh/km)
CHG_EFFICIENCY   = 1       # charging efficiency

# ---------- 1. 轻量化转 pandas ----------
ce = charging_events[            # GPU → CPU，只要必要列
    ['taxiid', 'start_time', 'end_time',
     'nearest_station_id', 'duration_s']].copy().to_pandas()

stays = final_stays[             # GPU → CPU，只要必要列
    ['taxiid', 'start_time', 'end_time', 'duration_s']].copy().to_pandas()

stations = stations_df[['station_id', 'avg_power']].copy() # 站点平均功率


In [ ]:
# ---------- 2. 上一次充电信息 ----------
ce = ce.sort_values(['taxiid', 'start_time'])
ce['prev_end'] = ce.groupby('taxiid')['end_time'].shift()
ce['gap_s']    = (ce['start_time'] - ce['prev_end']).dt.total_seconds()

# 仅对 gap_s 非空（= 不是首充电）的行建立区间
intervals = (
    ce[ce['gap_s'].notna()][['taxiid', 'prev_end', 'start_time']]
        .copy()
        .rename(columns={
            'prev_end'  : 'int_start',   # 区间起点
            'start_time': 'int_end'      # 区间终点
        })
)

In [ ]:
# ---------- 2. 上一次充电信息 ----------
# ce = ce.sort_values(['taxiid', 'start_time'])
# ce['prev_end'] = ce.groupby('taxiid')['end_time'].shift()
# ce['gap_s']    = (ce['start_time'] - ce['prev_end']).dt.total_seconds()
#
# # ---------- 3. 非充电停留总时长 idle_s ----------
# #   3.1 仅保留有 gap 的区间行
# intervals = ce[ce['gap_s'].notna()][['taxiid', 'prev_end', 'start_time']].copy()
# intervals.rename(columns={'prev_end': 'int_start',
#                           'start_time': 'int_end'}, inplace=True)
#
# #   3.2 merge stays ⇆ intervals → 布尔过滤
# tmp = (stays.merge(intervals, on='taxiid', how='inner')
#              .query('start_time >= int_start and end_time <= int_end'))
#
# idle_df = (tmp.groupby(['taxiid', 'int_start'], sort=False)
#                ['duration_s'].sum().reset_index())
#
# idle_df.rename(columns={'duration_s': 'idle_s'}, inplace=True)
# ce = ce.merge(idle_df,
#               left_on=['taxiid', 'prev_end'],
#               right_on=['taxiid', 'int_start'],
#               how='left') \
#        .drop(columns='int_start')
#
# ce['idle_s'] = ce['idle_s'].fillna(0)

In [ ]:
stays_sorted = (
    stays.dropna(subset=['taxiid', 'start_time', 'end_time'])
         .sort_values(['taxiid', 'start_time'], kind='mergesort')
)
intervals_sorted = (
    intervals.dropna(subset=['taxiid', 'int_start', 'int_end'])
             .sort_values(['taxiid', 'int_start'], kind='mergesort')
)
intervals_sorted.head(10)

In [ ]:
print(intervals_sorted[intervals_sorted['taxiid']='UUUBDZ9542'])

In [ ]:
test3=intervals_sorted['int_start'].isna().any()
test3

In [ ]:
def has_break(s):
    return (~s.is_monotonic_increasing)

bad_taxis = intervals_sorted.groupby('taxiid')['int_start'].apply(has_break)
print(bad_taxis[bad_taxis].index.tolist()[:10])   # 列出前几辆有问题的车

In [ ]:

# 0️⃣  已有的 ce / stays / intervals
# ce          : 充电事件（含 prev_end, gap_s 已算好）
# intervals   : ['taxiid','int_start','int_end']  ← 由 ce 派生
# stays       : ['taxiid','start_time','end_time','duration_s']


# 1️⃣  确保按 taxiid + 时间排序（merge_asof 要求升序）


intervals_sorted = (
    intervals.dropna(subset=['taxiid', 'int_start', 'int_end'])
             .sort_values(['taxiid', 'int_start'], kind='mergesort')
)
# 2️⃣  merge_asof  : 对每个停留  ↔ 最近的不晚于它的 interval 起点
merged = pd.merge_asof(
    stays_sorted,                # left
    intervals_sorted,            # right
    left_on='start_time',
    right_on='int_start',
    by='taxiid',
    direction='backward',        # 取“<=” 匹配
    allow_exact_matches=True
)

# 3️⃣  过滤：停留必须完全落在 interval 内部
mask = merged['end_time'] <= merged['int_end']
valid_idle = merged.loc[mask, ['taxiid', 'int_start', 'duration_s']]

# 4️⃣  汇总：每 (taxiid, int_start) 累加 idle_s
idle_df = (valid_idle
           .groupby(['taxiid', 'int_start'], sort=False)
           ['duration_s'].sum()
           .reset_index()
           .rename(columns={'duration_s': 'idle_s'}))

# 5️⃣  回并到 ce   (右键 = prev_end ≡ int_start)
ce = (ce.merge(idle_df,
               left_on=['taxiid', 'prev_end'],
               right_on=['taxiid', 'int_start'],
               how='left')
        .drop(columns='int_start'))

ce['idle_s'] = ce['idle_s'].fillna(0)

In [ ]:
# ---------- 4. 行驶时间 / 距离 / 耗电 ----------
ce['drive_s']  = (ce['gap_s'] - ce['idle_s']).clip(lower=0)
ce['distance_km']     = ce['drive_s']/3600 * AVG_SPEED_KMH
ce['energy_used_kWh'] = ce['distance_km'] * CONS_KWH_PER_KM

# ---------- 5. 充入电量 ----------
ce = ce.merge(stations, left_on='nearest_station_id',
                        right_on='station_id', how='left')
ce.drop(columns='station_id', inplace=True)

ce['energy_in_kWh'] = ce['charge_s']/3600 * ce['avg_power'] * CHG_EFFICIENCY
ce.drop(columns='avg_power', inplace=True)

In [ ]:
# ---------- 6. 逐车递推 SOC ----------
def soc_trace(group):
    soc_b, soc_a = [], []
    soc_prev = CAP_KWH                                # 初始满电
    for _, row in group.iterrows():
        soc_before = max(0, soc_prev - row.energy_used_kWh)
        soc_after  = min(CAP_KWH, soc_before + row.energy_in_kWh)
        soc_b.append(soc_before)
        soc_a.append(soc_after)
        soc_prev = soc_after
    group['SOC_before_kWh'] = soc_b
    group['SOC_after_kWh']  = soc_a
    return group

ce = (ce.groupby('taxiid', sort=False, group_keys=False)
         .apply(soc_trace))

# 可选：百分比形式
ce['SOC_before_pct'] = ce['SOC_before_kWh'] / CAP_KWH * 100
ce['SOC_after_pct']  = ce['SOC_after_kWh']  / CAP_KWH * 100

# ---------- 7. 保存 / 返回 ----------
cols_order = [
    'taxiid', 'start_time', 'end_time',
    'SOC_before_kWh', 'SOC_after_kWh',
    'energy_used_kWh', 'energy_in_kWh',
    'distance_km', 'drive_s', 'idle_s', 'gap_s'
]
battery_trace_df = ce[cols_order].copy()
battery_trace_df.to_parquet('battery_trace.parquet', index=False)


# Leaving station

In [ ]:
def find_passenger_pickup_batch(charging_events, db_url, table_name, batch_size=100):
    """
    分批查询：每次查询100辆车 - 平衡速度和内存
    """
    charge_df = charging_events[['taxiid', 'end_time']].to_pandas()
    unique_taxis = charge_df['taxiid'].unique()

    # 分批处理
    n_batches = int(np.ceil(len(unique_taxis) / batch_size))

    all_results = []

    conn = psycopg2.connect(db_url)

    for i in range(n_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(unique_taxis))
        batch_taxis = unique_taxis[start_idx:end_idx]

        # 这批车辆的充电事件
        batch_charges = charge_df[charge_df['taxiid'].isin(batch_taxis)]

        if len(batch_charges) == 0:
            continue

        min_time = batch_charges['end_time'].min()
        max_time = batch_charges['end_time'].max() + pd.Timedelta(minutes=10)

        # 批量查询这批车辆
        query = f"""
        SELECT taxiid, time
        FROM {table_name}
        WHERE passenger > 0
          AND taxiid IN ({','.join([f"'{tid}'" for tid in batch_taxis])})
          AND time > '{min_time}'
          AND time <= '{max_time}'
        ORDER BY taxiid, time
        """

        pickup_data = pd.read_sql(query, conn)

        # 内存中匹配
        for _, charge_row in batch_charges.iterrows():
            taxiid = charge_row['taxiid']
            charge_end = charge_row['end_time']
            look_until = charge_end + pd.Timedelta(minutes=10)

            taxi_pickups = pickup_data[
                (pickup_data['taxiid'] == taxiid) &
                (pickup_data['time'] > charge_end) &
                (pickup_data['time'] <= look_until)
            ]

            if len(taxi_pickups) > 0:
                first_pickup = taxi_pickups['time'].min()
                minutes_after = (first_pickup - charge_end).total_seconds() / 60
                all_results.append({
                    'taxiid': taxiid,
                    'charge_end_time': charge_end,
                    'pickup_time': first_pickup,
                    'minutes_after_charging': minutes_after,
                    'found': True
                })
            else:
                all_results.append({
                    'taxiid': taxiid,
                    'charge_end_time': charge_end,
                    'pickup_time': None,
                    'minutes_after_charging': None,
                    'found': False
                })

        # 清理内存
        del pickup_data
        print(f"Processed batch {i+1}/{n_batches}")

    conn.close()
    return pd.DataFrame(all_results)

In [ ]:
test_events=charging_events.head(20)

In [ ]:
result_df = find_passenger_pickup_batch(test_events, DB_URL, table_name, batch_size=20)

In [ ]:
result_df

In [ ]:
def find_passenger_pickup_sql_join(charging_events, db_url, table_name):
    """
    使用SQL JOIN一次性匹配 - 让数据库做优化
    """
    import psycopg2
    import pandas as pd
    from io import StringIO

    # 准备充电事件数据
    charge_df = charging_events[['taxiid', 'end_time']].to_pandas()
    charge_df['look_end_time'] = charge_df['end_time'] + pd.Timedelta(minutes=10)
    # 重命名列以匹配临时表
    charge_df = charge_df.rename(columns={'end_time': 'charge_end_time'})

    conn = psycopg2.connect(db_url)
    cur = conn.cursor()

    # 创建临时表（一次性写入）
    cur.execute("""
        CREATE TEMP TABLE charging_ends (
            taxiid VARCHAR,
            charge_end_time TIMESTAMP,
            look_end_time TIMESTAMP
        )
    """)

    # 批量插入（快速）
    buffer = StringIO()
    # 只输出需要的列，顺序要匹配
    charge_df[['taxiid', 'charge_end_time', 'look_end_time']].to_csv(
        buffer, index=False, header=False, sep='\t'
    )
    buffer.seek(0)
    cur.copy_from(
        buffer,
        'charging_ends',
        columns=['taxiid', 'charge_end_time', 'look_end_time']
    )

    # 一次性JOIN查询（数据库优化）
    query = """
    SELECT
        ce.taxiid,
        ce.charge_end_time,
        MIN(gps.time) as pickup_time,
        EXTRACT(EPOCH FROM (MIN(gps.time) - ce.charge_end_time)) / 60 as minutes_after
    FROM charging_ends ce
    LEFT JOIN {table_name} gps
        ON gps.taxiid = ce.taxiid
        AND gps.passenger = 1
        AND gps.time > ce.charge_end_time
        AND gps.time <= ce.look_end_time
    GROUP BY ce.taxiid, ce.charge_end_time
    ORDER BY ce.taxiid, ce.charge_end_time
    """.format(table_name=table_name)

    results = pd.read_sql(query, conn)

    # 清理临时表
    cur.execute("DROP TABLE charging_ends")
    conn.commit()
    conn.close()

    # 处理结果：如果没有匹配到，minutes_after会是NULL
    results['found'] = results['pickup_time'].notna()
    results['minutes_after_charging'] = results['minutes_after']

    return results

In [ ]:
test_events2=charging_events.head(10)

In [ ]:
result_df2 = find_passenger_pickup_sql_join(test_events2, DB_URL, table_name)

# Visualization

In [ ]:
# --- 分析: 平均每日充电次数 ---

print("--- 正在计算平均每日充电次数 ---")

if 'char_que_df' in locals() and not char_que_df.empty:

    # --- 1. 提取充电日期 ---
    # .dt.date 用于从 datetime 对象中仅提取日期部分
    char_que_df['charge_date'] = char_que_df['arrive_time'].dt.date

    # --- 2. 计算每辆车每天的成功充电次数 ---
    # 我们只统计没有放弃 ('giveup' == False) 的充电事件
    # daily_charges_per_taxi = char_que_df[char_que_df['giveup'] == False].groupby(['taxiid', 'charge_date']).size().reset_index
    daily_charges_per_taxi = char_que_df.groupby(['taxiid', 'charge_date']).size().reset_index(name='daily_charge_count')
    print("\n--- 每辆车每日充电次数 (示例) ---")
    print(daily_charges_per_taxi.head())

    # --- 3. 计算每辆车的“平均每日充电次数” ---
    # 对上一步的结果再次 groupby，这次只按 taxiid，然后求均值
    avg_daily_charges_per_taxi = daily_charges_per_taxi.groupby('taxiid')['daily_charge_count'].mean().reset_index(name='avg_daily_charges')
    print("\n--- 每辆车的平均每日充电次数 (示例) ---")
    print(avg_daily_charges_per_taxi.head())

    # --- 4. 计算所有车的总平均每日充电次数 ---
    # 直接对 'avg_daily_charges' 这一列求均值
    overall_avg_daily_charges = avg_daily_charges_per_taxi['avg_daily_charges'].mean()
    print("\n" + "="*40)
    print(f"✅ 所有车辆的平均每日充电次数为: {overall_avg_daily_charges:.2f} 次/天/车")
    print("="*40)

else:
    print("错误: 'char_que_df' 未找到或为空。请先运行队列模拟部分的单元格。")


In [ ]:
# --- 分析: 停留与等待时间的分布统计 ---

import numpy as np

print("--- 正在计算停留与等待时间的描述性统计 ---")

if 'char_que_df' in locals() and not char_que_df.empty:
    # --- 1. 准备数据 ---
    # 如果 'stay_duration_s' 不存在，则从时间戳计算
    if 'stay_duration_s' not in char_que_df.columns:
        char_que_df['stay_duration_s'] = (char_que_df['leave_time'] - char_que_df['arrive_time']).dt.total_seconds()

    # 为了更好地可视化和理解，我们将秒转换为分钟
    char_que_df['wait_minutes'] = char_que_df['wait_dur'] / 60
    char_que_df['stay_duration_minutes'] = char_que_df['stay_duration_s'] / 60

    # --- 2. 输出描述性统计 ---
    print("\n--- 排队时间 (分钟) 的描述性统计 ---")
    # 使用 .describe() 查看分布，特别是 50% (中位数), 75%, max 等
    # percentiles 参数可以自定义需要查看的百分位数
    print(char_que_df['wait_minutes'].describe(percentiles=[.25, .5, .75, .9, .95, .99]))

    print("\n--- 总停留时长 (分钟) 的描述性统计 ---")
    print(char_que_df['stay_duration_minutes'].describe(percentiles=[.25, .5, .75, .9, .95, .99]))

else:
    print("错误: 'char_que_df' 未找到或为空。请先运行队列模拟部分的单元格。")

In [ ]:
# --- 可视化: 停留与等待时间的分布直方图 ---

import matplotlib.pyplot as plt
import seaborn as sns

print("--- 正在生成停留与等待时间的分布直方图 ---")

if 'char_que_df' in locals() and 'wait_minutes' in char_que_df.columns:
    # 设置绘图风格
    sns.set_style("whitegrid")

    # 创建一个 1x2 的子图布局
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('Distribution of Waiting and Staying Durations', fontsize=16)

    # a) 排队时间直方图
    # 为了使图形更有意义，我们只关注那些实际需要排队的事件 (wait_minutes > 0)
    waiting_events_df = char_que_df[char_que_df['wait_minutes'] > 0.1] # 使用0.1避免浮点数精度问题
    sns.histplot(data=waiting_events_df, x='wait_minutes', bins=50, ax=axes[0], kde=True)
    axes[0].set_title('Waiting Time Distribution (for events with wait > 0)')
    axes[0].set_xlabel('Waiting Time (minutes)')
    axes[0].set_ylabel('Number of Events')
    # 限制 x 轴范围以便更好地观察主要分布，例如最多看 2 小时
    axes[0].set_xlim(0, 120)

    # b) 总停留时长直方图
    sns.histplot(data=char_que_df, x='stay_duration_minutes', bins=50, ax=axes[1], color='skyblue', kde=True)
    axes[1].set_title('Total Stay Duration Distribution')
    axes[1].set_xlabel('Total Stay Duration (minutes)')
    axes[1].set_ylabel('Number of Events')
    # 限制 x 轴范围，例如最多看 3 小时
    axes[1].set_xlim(0, 180)

    # 调整布局以防止标题重叠并显示图像
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

else:
    print("错误: 'char_que_df' 或 'wait_minutes' 列未找到。请先运行前面的统计单元格。")

In [ ]:

# 1. 准备数据（都在 CPU）
stations_pdf = pd.read_csv('station_information.csv')  # 充电站
charging_pdf = charging_events.to_pandas()             # 充电行为（距离阈值内的停留点）

# 2. 计算地图中心（深圳大致中心 or 所有点的平均）
center_lat = charging_pdf['stay_lat'].mean()
center_lon = charging_pdf['stay_lon'].mean()

# 3. 创建底图
m = folium.Map(location=[center_lat, center_lon],
               zoom_start=11,
               tiles='OpenStreetMap')

# # 4. 充电站：用 MarkerCluster 以免过度遮挡
# station_cluster = MarkerCluster(name='Charging Stations').add_to(m)
# for _, row in stations_pdf.iterrows():
#     folium.CircleMarker(
#         location=[row['latitude'], row['longitude']],
#         radius=4,
#         color='blue',
#         fill=True,
#         fill_color='blue',
#         fill_opacity=0.7,
#         popup=f"Station ID: {row['station_id']}"
#     ).add_to(station_cluster)

# 5. 充电行为：红色半透明圆
charge_cluster = MarkerCluster(name='Charging Events').add_to(m)
for _, row in charging_pdf.iterrows():
    folium.CircleMarker(
        location=[row['stay_lat'], row['stay_lon']],
        radius=6,                      # 可改为 min(row['duration_s']/60, 10) 表示时长
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.5,
        popup=(f"TaxiID: {row['taxiid']}<br>"
               f"Start: {row['start_time']}<br>"
               f"End: {row['end_time']}<br>"
               f"Duration: {row['duration_s']} s<br>"
               f"Dist to Station: {row['distance_to_station_m']:.1f} m<br>"
               f"Station ID: {row['nearest_station_id']}")
    ).add_to(charge_cluster)

# 6. 图例/图层控制
folium.LayerControl().add_to(m)

# 7. 显示
m